In [ ]:
from langchain_core.example_selectors import LengthBasedExampleSelector, SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
from typing import List
import volcenginesdkarkruntime
from volcenginesdkarkruntime import Ark

class DoubaoEmbeddings(Embeddings):
    """火山引擎方舟embedding模型封装"""
    
    def __init__(self, model_name: str = "doubao-embedding-text-240715"):
        self.model_name = model_name
        self.client = Ark(api_key=api_key)
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """对文档列表进行向量化"""
        embeddings = []
        for text in texts:
            # 调用方舟API获取embedding
            response = self.client.embeddings.create(
                model=self.model_name,
                input=text
            )
            embeddings.append(response.data[0].embedding)
        return embeddings
    
    def embed_query(self, text: str) -> List[float]:
        """对查询文本进行向量化"""
        return self.embed_documents([text])[0]

/Users/bytedance/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tail", "output": "head"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input}\n反义词：{output}"
)

In [3]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义词：",
    input_variables=["adjective"]
)
print(dynamic_prompt.format(adjective="big and huge adn massive and large and gigantic and tall and much much much much much much bigger then everyone"))

给出每个输入词的反义词

原词：happy
反义词：sad

原词：tail
反义词：head

原词：big and huge adn massive and large and gigantic and tall and much much much much much much bigger then everyone
反义词：


In [4]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  
    embeddings, 
    FAISS,
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义词：",
    input_variables=["adjective"]
)
print(similar_prompt.format(adjective="晴朗"))

给出每个输入词的反义词

原词：高兴
反义词：悲伤

原词：晴朗
反义词：


In [5]:
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = DoubaoEmbeddings()

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,  
    embeddings, 
    Chroma,
    k=2,
)
mar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义词：",
    input_variables=["adjective"]
)
print(mar_prompt.format(adjective="难过"))

给出每个输入词的反义词

原词：happy
反义词：sad

原词：tail
反义词：head

原词：难过
反义词：


In [6]:
print(sy)

NameError: name 'sy' is not defined